## (0) Libraries 📚

In [1]:
import pandas as pd
from thefuzz import fuzz, process
from concurrent.futures import ThreadPoolExecutor, as_completed, ProcessPoolExecutor
import os
import time

# 🤖 Recipes data


In [3]:
# load recipes dataset
df_recipes = pd.read_csv("recipes_ingredients.csv",nrows=50)

In [4]:
df_recipes.shape

(50, 9)

In [5]:
df_recipes.head(10)

,id,name,description,ingredients,ingredients_raw,steps,servings,serving_size,tags
0,71247,Cherry Streusel Cobbler,"I haven't made this in years, so I'm just gues...","[""cherry pie filling"", ""condensed milk"", ""melt...","[""2 (21 ounce) cans cherry pie filling"",""2...","[""Preheat oven to 375°F."", ""Spread cherry pie ...",6.0,1 (347 g),"[""60-minutes-or-less"", ""time-to-make"", ""course..."
1,76133,Reuben and Swiss Casserole Bake,I think this is even better than a reuben sand...,"[""corned beef chopped"", ""sauerkraut cold water...","[""1/2-1 lb corned beef, cooked and choppe...","[""Set oven to 350 degrees F."", ""Butter a 9 x 1...",4.0,1 (207 g),"[""60-minutes-or-less"", ""time-to-make"", ""course..."
2,503816,Yam-Pecan Recipe,A lady I work with heard me taking about ZWT a...,"[""unsalted butter"", ""vegetable oil"", ""all - pu...","[""3/4 cup unsalted butter, at room tempera...","[""Preheat oven to 350°F In a mixing bowl, usi...",8.0,1 (198 g),"[""time-to-make"", ""course"", ""main-ingredient"", ..."
3,418749,Tropical Orange Layer Cake,An easy and delicious cake. Great for a summ...,"[""orange cake mix"", ""instant vanilla pudding"",...","[""1 (18 ounce) pkge.orange cake mix"",""1 (3...","[""In a large mixing bowl, combine the first 6 ...",16.0,1 (191 g),"[""60-minutes-or-less"", ""time-to-make"", ""course..."
4,392934,Safe to Eat Raw Chocolate Chip Oreo Cookie &qu...,I was searching the web for something like thi...,"[""butter"", ""brown sugar"", ""granulated sugar"", ...","[""1/2 cup butter, room temperature "",""1/2 ...","[""Cream butter and sugars together."", ""Blend i...",24.0,1 (26 g),"[""15-minutes-or-less"", ""time-to-make"", ""course..."
5,532245,Chicken and Petite Carrots,I made this for dinner since I had nothing to ...,"[""chicken breasts cutlets"", ""petite carrots"", ...","[""2 chicken breasts, cutlets "",""1 (2 l...","[""Step 1: Pull out a cutting board and cut the...",4.0,1 (443 g),"[""time-to-make"", ""preparation"", ""presentation""..."
6,489452,Teriyaki Pork Chops,I made these on a whim and they are my husband...,"[""teriyaki sauce"", ""pork chops""]","[""1 (16 ounce) bottle teriyaki sauce"",""4 ...","[""I like to marinade them overnight in a ziplo...",4.0,1 (313 g),"[""weeknight"", ""15-minutes-or-less"", ""time-to-m..."
7,126368,Bobbie's Pie Crust,"Another variation on the ""don't have to roll i...","[""flour""]","[""2 cups flour"",""2 tablespoons sugar...","[""Mix all ingredients well and pat into pie pa...",8.0,1 (58 g),"[""30-minutes-or-less"", ""time-to-make"", ""course..."
8,306467,Quick Bolognese Sauce,Quick Bolognese Sauce Recipe from Festival Foo...,"[""olive oil"", ""yellow onion"", ""celery rib"", ""c...","[""2 tablespoons light olive oil"",""1 sma...","[""Saute onion, celery, and carrot in olive oil...",6.0,1 (297 g),"[""time-to-make"", ""course"", ""main-ingredient"", ..."
9,318331,Granny's Butter Rolls,"Recipe from Crystal Van Poppe in her booklet ""...","[""biscuit mix"", ""granulated sugar"", ""butter"", ...","[""2 1/4 cups biscuit mix"",""2/3 cup wat...","[""Mix biscuit mix and water until a soft dough...",12.0,1 (91 g),"[""60-minutes-or-less"", ""time-to-make"", ""course..."


In [6]:
# input recipe

recette = "Quick Bolognese Sauce"

In [7]:
# load recipe ingredients

list_ingredients = df_recipes[df_recipes["name"] == recette]["ingredients"].iloc[0]
print(list_ingredients)

["olive oil", "yellow onion", "celery rib", "carrot chopped", "garlic salt", "black pepper", "tomato paste", "dry red wine", "beef broth", "crushed tomatoes", "heavy cream", "fresh basil", "parmesan cheese"]


In [8]:
list_ingredients = list_ingredients.strip('["]').replace('", "', ', ')
list_ingredients = list_ingredients.split(", ")

In [9]:
list_ingredients

['olive oil',
 'yellow onion',
 'celery rib',
 'carrot chopped',
 'garlic salt',
 'black pepper',
 'tomato paste',
 'dry red wine',
 'beef broth',
 'crushed tomatoes',
 'heavy cream',
 'fresh basil',
 'parmesan cheese']

# 🤖 Y-trust


##  (1) Original dataset 💾  - Done

In [ ]:
# count the # of rows in the dataset
with open("../../dataset_y-trust/en.openfoodfacts.org.products.csv", "r", encoding="utf-8") as f:
    row_count = sum(1 for line in f) - 1  # -1 to exclude the header
print(row_count)

3871738


In [8]:
df = pd.read_csv("../../dataset_y-trust/en.openfoodfacts.org.products.csv", on_bad_lines='skip' , sep="\t", nrows = 1200000)

/tmp/ipykernel_80869/3261988283.py:1: DtypeWarning: Columns (0,11,17,27,28,32,33,34,36,54,68,73) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../../dataset_y-trust/en.openfoodfacts.org.products.csv", on_bad_lines='skip' , sep="\t", nrows = 1200000)


In [9]:
pd.set_option("display.max_columns", None)

In [22]:
df.head(1)

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,last_modified_by,last_updated_t,last_updated_datetime,product_name,abbreviated_product_name,generic_name,quantity,packaging,packaging_tags,packaging_en,packaging_text,brands,brands_tags,brands_en,categories,categories_tags,categories_en,origins,origins_tags,origins_en,manufacturing_places,manufacturing_places_tags,labels,labels_tags,labels_en,emb_codes,emb_codes_tags,first_packaging_code_geo,cities,cities_tags,purchase_places,stores,countries,countries_tags,countries_en,ingredients_text,ingredients_tags,ingredients_analysis_tags,allergens,allergens_en,traces,traces_tags,traces_en,serving_size,serving_quantity,no_nutrition_data,additives_n,additives,additives_tags,additives_en,nutriscore_score,nutriscore_grade,nova_group,pnns_groups_1,pnns_groups_2,food_groups,food_groups_tags,food_groups_en,states,states_tags,states_en,brand_owner,environmental_score_score,environmental_score_grade,nutrient_levels_tags,product_quantity,owner,data_quality_errors_tags,unique_scans_n,popularity_tags,completeness,last_image_t,last_image_datetime,main_category,main_category_en,image_url,image_small_url,image_ingredients_url,image_ingredients_small_url,image_nutrition_url,image_nutrition_small_url,energy-kj_100g,energy-kcal_100g,energy_100g,energy-from-fat_100g,fat_100g,saturated-fat_100g,butyric-acid_100g,caproic-acid_100g,caprylic-acid_100g,capric-acid_100g,lauric-acid_100g,myristic-acid_100g,palmitic-acid_100g,stearic-acid_100g,arachidic-acid_100g,behenic-acid_100g,lignoceric-acid_100g,cerotic-acid_100g,montanic-acid_100g,melissic-acid_100g,unsaturated-fat_100g,monounsaturated-fat_100g,omega-9-fat_100g,polyunsaturated-fat_100g,omega-3-fat_100g,omega-6-fat_100g,alpha-linolenic-acid_100g,eicosapentaenoic-acid_100g,docosahexaenoic-acid_100g,linoleic-acid_100g,arachidonic-acid_100g,gamma-linolenic-acid_100g,dihomo-gamma-linolenic-acid_100g,oleic-acid_100g,elaidic-acid_100g,gondoic-acid_100g,mead-acid_100g,erucic-acid_100g,nervonic-acid_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,added-sugars_100g,sucrose_100g,glucose_100g,fructose_100g,galactose_100g,lactose_100g,maltose_100g,maltodextrins_100g,starch_100g,polyols_100g,erythritol_100g,fiber_100g,soluble-fiber_100g,insoluble-fiber_100g,proteins_100g,casein_100g,serum-proteins_100g,nucleotides_100g,salt_100g,added-salt_100g,sodium_100g,alcohol_100g,vitamin-a_100g,beta-carotene_100g,vitamin-d_100g,vitamin-e_100g,vitamin-k_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,vitamin-b6_100g,vitamin-b9_100g,folates_100g,vitamin-b12_100g,biotin_100g,pantothenic-acid_100g,silica_100g,bicarbonate_100g,potassium_100g,chloride_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,fluoride_100g,selenium_100g,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,fruits-vegetables-nuts-dried_100g,fruits-vegetables-nuts-estimate_100g,fruits-vegetables-nuts-estimate-from-ingredients_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,carbon-footprint-from-meat-or-fish_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g,choline_100g,phylloquinone_100g,beta-glucan_100g,inositol_100g,carnitine_100g,sulphate_100g,nitrate_100g,acidity_100g,carbohydrates-total_100g
0,54,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1582569031,2020-02-24T18:30:31Z,1733085204,2024-12-01T20:33:24Z,NaN,1.740205e+09,2025-02-22T06:23:42Z,Limonade artisanale a la rose,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en:fr,en:france,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unknown,NaN,unknown,unknown,NaN,NaN,NaN,"en:to-be-completed, en:nutrition-facts-to-be-c...","en:to-be-completed,en:nutrition-facts-to-be-co...","To be completed,Nutrition facts to be com

In [20]:
nutella = df["url"][28]

In [21]:
nutella

'http://world-en.openfoodfacts.org/product/00000023/mezzo-mix-coca-cola'

##  (1.1) Data exploration ℹ️  - Done

In [59]:
# calcul des % de colonnes vides
colonnes = [
    "product_name",
    "origins_en",
    "origins_tags",
    "manufacturing_places",
    "manufacturing_places_tags",
    "first_packaging_code_geo",
    "cities",
    "cities_tags",
    "purchase_places",
    "stores",
    "countries",
    "countries_en",
    "countries_tags",
    "serving_size",
    "labels",
    "labels_en",
    "labels_tags",
    "image_url",
    "image_small_url",
    "last_image_t",
    "brand_owner",
    "emb_codes",
    "emb_codes_tags",
    "pnns_groups_1",
    "pnns_groups_2",
    "food_groups",
    "serving_quantity",
    "ingredients_text",
    "ingredients_tags",
    "ingredients_analysis_tags"
]

# Calcul du % de NaN par colonne
pct_nan = df[colonnes].isna().mean() * 100

# Affichage avec arrondi à 2 décimales
print(pct_nan.round(2))

product_name                   4.65
origins_en                    98.24
origins_tags                  98.24
manufacturing_places          98.12
manufacturing_places_tags     98.12
first_packaging_code_geo      99.26
cities                       100.00
cities_tags                   99.18
purchase_places               97.45
stores                        93.13
countries                      0.53
countries_en                   0.53
countries_tags                 0.53
serving_size                  52.99
labels                        77.99
labels_en                     77.99
labels_tags                   77.98
image_url                     41.16
image_small_url               41.16
last_image_t                  37.65
brand_owner                   75.37
emb_codes                     98.36
emb_codes_tags                98.36
pnns_groups_1                  0.41
pnns_groups_2                  0.41
food_groups                   69.76
serving_quantity              53.45
ingredients_text            

In [14]:
country = "France"
col = "countries"

# Filtrer par pays
df_country = df[df[col] == country]

# Sélectionner colonnes d’intérêt
colonnes = [
    "pnns_groups_1",
    "pnns_groups_2",
    "environmental_score_score",
    "completeness",
    "quantity",
    "ingredients_text",
    "ingredients_tags",
    "ingredients_analysis_tags"
]
percent_missing = df_country[colonnes].isna().mean() * 100

print(percent_missing.round(2))


pnns_groups_1                 0.00
pnns_groups_2                 0.00
environmental_score_score    53.22
completeness                  0.01
quantity                     67.48
ingredients_text             68.04
ingredients_tags             68.10
ingredients_analysis_tags    67.35
dtype: float64


In [ ]:
# displaying data only for a given country

df_code = df_country[df_country["first_packaging_code_geo"].notna()]
pd.set_option("display.max_rows", None)
df_code.head(10)


In [87]:
url = df["image_url"][39]
url_small = df["image_small_url"][39]

In [88]:
url

'https://images.openfoodfacts.org/images/products/000/000/000/0034/front_en.25.400.jpg'

In [89]:
url_small

'https://images.openfoodfacts.org/images/products/000/000/000/0034/front_en.25.200.jpg'

##  (1.2) Reducing dataset size - Done

In [ ]:
# reducing dataframe to given set of columns (local df)

columns_kept1 = ["code", "product_name", "pnns_groups_2", "brands", "quantity", "image_small_url",
"energy-kcal_100g", "carbohydrates_100g", "proteins_100g", "fat_100g",
"first_packaging_code_geo", "emb_codes_tags", "emb_codes_tags",
"countries", "countries_en", "countries_tags", "origins", "origins_en",
"origins_tags", "manufacturing_places", "manufacturing_places_tags", "first_packaging_code_geo", "cities", "cities_tags", "purchase_places", "stores",
]

df_select = df[columns_kept1]

In [ ]:
# generate csv from the selected columns

df_select.to_csv("open_food_df_selected_columns.csv", sep="\t", index=False, encoding="utf-8")

In [ ]:
# extracting the original dataset with only select set of columns as a csv

columns_kept1 = ["code", "product_name", "pnns_groups_2", "brands", "quantity", "image_small_url",
"energy-kcal_100g", "carbohydrates_100g", "proteins_100g", "fat_100g",
"first_packaging_code_geo", "emb_codes_tags", "emb_codes_tags",
"countries", "countries_en", "countries_tags", "origins", "origins_en",
"origins_tags", "manufacturing_places", "manufacturing_places_tags", "first_packaging_code_geo", "cities", "cities_tags", "purchase_places", "stores",
]

# Create an iterator that reads the file in chunks
chunks = pd.read_csv("../../dataset_y-trust/en.openfoodfacts.org.products.csv",sep="\t", usecols=columns_kept1, chunksize=100000)

# Open a new file and write the filtered data into it chunk by chunk
with open("open_food_df_selected_columns_v2.csv", "w", encoding="utf-8", newline="") as f_out:
    for i, chunk in enumerate(chunks):
        chunk.to_csv(f_out, sep="\t", index=False, header=(i == 0))  # write header only once

/tmp/ipykernel_32829/2088301801.py:15: DtypeWarning: Columns (33,34,36) have mixed types. Specify dtype option on import or set low_memory=False.
  for i, chunk in enumerate(chunks):
/tmp/ipykernel_32829/2088301801.py:15: DtypeWarning: Columns (33,34,36) have mixed types. Specify dtype option on import or set low_memory=False.
  for i, chunk in enumerate(chunks):
/tmp/ipykernel_32829/2088301801.py:15: DtypeWarning: Columns (33,34,36) have mixed types. Specify dtype option on import or set low_memory=False.
  for i, chunk in enumerate(chunks):
/tmp/ipykernel_32829/2088301801.py:15: DtypeWarning: Columns (27,28,33,34,36) have mixed types. Specify dtype option on import or set low_memory=False.
  for i, chunk in enumerate(chunks):
/tmp/ipykernel_32829/2088301801.py:15: DtypeWarning: Columns (24,25,26,27,28,33,34,36,38) have mixed types. Specify dtype option on import or set low_memory=False.
  for i, chunk in enumerate(chunks):
/tmp/ipykernel_32829/2088301801.py:15: DtypeWarning: Columns 

In [23]:
# telechargement du dataset
df_temp = pd.read_csv("open_food_df_selected_columns_v2.csv", on_bad_lines='skip' , sep="\t")
pd.set_option("display.max_columns", None)


/tmp/ipykernel_80869/4164841631.py:2: DtypeWarning: Columns (0,11,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df_temp = pd.read_csv("open_food_df_selected_columns_v2.csv", on_bad_lines='skip' , sep="\t")


In [ ]:
# preview des différentes valeurs d'une colonne

for value in df_temp['countries_en'].unique():
    print(value)

France
Germany
Germany,Spain
Brazil,Germany
Germany,Greece
France,Germany
France,Germany,United States
Germany,United States
Germany,World
Switzerland
nan
France,Ireland
Germany,Ireland
Brazil
Belgium,Germany
Spain
United States
European Union
Selestosina
Ireland
France,Spain
United Kingdom
France,United Kingdom
Moldova
Italy
Belgium
Egypt
Comidosil
Exland
France,Gabon,Germany
Germany,Malaysia
Germany,Italy
Germany,United Kingdom
France,United States
Germany,Mexico
Austria
Mexico
Romania
Austria,Irland,Deutschland
Turkey
Belgium,France,Germany
Germany,South Africa
Australia
Morocco
Italy,Spain
Canada
Norway
French Polynesia
United Arab Emirates
World
France,Ireland,United Kingdom
Qatar
Germany,India
Ukraine
India
Testeste
Iraq
Germany,Morocco
Bahrain
Panama
Senegal
Oman
Argentina
United Kingdom,World
France,World
Czech Republic
Malaysia
Germany,Ireland,World
South Africa
Sweden
Taiwan
Pakistan
Argentina,France
Poland
Belgium,United States
France,Allemagne
Lithuania
Cambodia
Thailand
Ph

In [24]:
# visualisation du dataset
df_temp.head(5)

,code,product_name,quantity,brands,origins,origins_tags,origins_en,manufacturing_places,manufacturing_places_tags,emb_codes_tags,first_packaging_code_geo,cities,cities_tags,purchase_places,stores,countries,countries_tags,countries_en,pnns_groups_2,image_small_url,energy-kcal_100g,fat_100g,carbohydrates_100g,proteins_100g
0,54,Limonade artisanale a la rose,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en:fr,en:france,France,unknown,https://images.openfoodfacts.org/images/produc...,NaN,NaN,NaN,NaN
1,63,CIABATTA OLIVE,NaN,EDEKA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en:fr,en:france,France,unknown,https://images.openfoodfacts.org/images/produc...,332.0,25.0,3.0,23.0
2,114,Chocolate n 3,80 g,Jeff de Bruges,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,France,en:france,France,unknown,https://images.openfoodfacts.org/images/produc...,NaN,44.0,30.0,7.1
3,1,Weihnachtsgelee (selbstgemacht von Jutta),280gr. 320 Kapseln,Bodylab,Spanien,en:spain,Spain,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DE,en:germany,Germany,unknown,https://images.openfoodfacts.org/images/produc...,374.0,2.0,7.0,79.0
4,105,Paleta gran reserva - Sierra nevada-,750ml,AdvoCare,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Spanien, Germany","en:germany,en:spain","Germany,Spain",Artificially sweetened beverages,https://images.openfoodfacts.org/images/produc...,NaN,NaN,NaN,NaN


In [ ]:
# Calcul du % de NaN par colonne dans le data set "final"
pct_nan = df_temp[columns_kept1].isna().mean() * 100

# Affichage avec arrondi à 2 décimales
print(pct_nan.round(2))

code                           0.00
product_name                   6.58
pnns_groups_2                  0.64
brands                        39.66
quantity                      67.29
image_small_url               25.02
energy-kcal_100g              27.16
carbohydrates_100g            26.61
proteins_100g                 26.59
fat_100g                      26.72
first_packaging_code_geo      97.65
emb_codes_tags                94.99
emb_codes_tags                94.99
countries                      0.60
countries_en                   0.60
countries_tags                 0.60
origins                       96.06
origins_en                    96.06
origins_tags                  96.06
manufacturing_places          94.60
manufacturing_places_tags     94.60
first_packaging_code_geo      97.65
cities                       100.00
cities_tags                   97.46
purchase_places               94.65
stores                        87.33
dtype: float64


In [117]:
country = "France"
col = "countries"

# Filtrer par pays
df_country = df_temp[df_temp[col] == country]

# Sélectionner colonnes d’intérêt
colonnes = [
    "origins_en",
    "origins",
    "manufacturing_places"
]
percent_missing = df_country[colonnes].isna().mean() * 100

print(percent_missing.round(2))

origins_en              89.13
origins                 89.13
manufacturing_places    86.62
dtype: float64


In [96]:
df_country.shape

(493923, 24)

##  (1.2) Pre-cleaning 🕵🏻‍♂️

* 👥 Remove duplicates
* ❓ Have a first glimpse at the missing values
* 😏 Detect the weird columns
* ⚖️ Detect class imbalances for classification tasks and outliers for regression tasks

In [ ]:
#filtres data completeness
# au moins chacune des colonnes nutritionnelles (energy-kcal_100g, carbohydrates_100g, proteins_100g, fat_100g)
# au moins countries de remplie
# dropouts : countries, energy-kcal_100g, carbohydrates_100g, proteins_100g, fat_100g, product_name
# duplicates

In [77]:
df_temp.shape

(3867740, 24)

In [78]:
# removing duplicates

df_dupl = df_temp.drop_duplicates()
df_dupl.shape

# we could also drop based on a specific column only (df_temp.dropèduplicates(subset=["column1","column2"]))

(3865703, 24)

In [ ]:
# dropouts : removing rows with no nutritional information

columns_to_check = ["energy-kcal_100g", "carbohydrates_100g", "proteins_100g", "fat_100g"]

df_clean_nutri = df_dupl.dropna(subset=columns_to_check, how="all")

df_clean_nutri.shape


(2870651, 24)

In [ ]:

# if checking only for composition columns:

columns_to_check_comp = ["carbohydrates_100g", "proteins_100g", "fat_100g"]

df_clean_nutri_comp = df_dupl.dropna(subset=columns_to_check_comp, how = "all")

df_clean_nutri_comp.shape

(2858464, 24)

In [80]:
# if checking that ALL 5 COLUMNS are filled:

columns_to_check = ["energy-kcal_100g", "carbohydrates_100g", "proteins_100g", "fat_100g"]

df_clean_nutri_all = df_dupl.dropna(subset=columns_to_check, how="any")

df_clean_nutri_all.shape

(2757260, 24)

In [ ]:
# dummy country of origin rule:
# first country in countries_en or origin

######df_cleaned_countries

# advanced version:
## trouver librairie qui interprete:
# créer une colonne #origin_country
### si field origins_en rempli on le prend
### sinon manufacturing_places
### sinon countries_en (premier pays)

In [ ]:
# dummy country list

col_test = df_clean_nutri_all["countries_en"].copy()
col_transformed = col_test.str.split(",").str[0]
comparison = pd.DataFrame({
    'avant': col_test,
    'après': col_transformed
})
print(comparison.head(10))

                           avant    après
1                         France   France
3                        Germany  Germany
5                 Brazil,Germany   Brazil
6                 Germany,Greece  Germany
7                 Brazil,Germany   Brazil
8                         France   France
9                 France,Germany   France
10  France,Germany,United States   France
12                 Germany,Spain  Germany
13         Germany,United States  Germany


In [81]:
# applying it to the df:

df_cleaned_countries = df_clean_nutri_all
df_cleaned_countries["countries_en"] = df_cleaned_countries["countries_en"].str.split(",").str[0].str.strip()
df_cleaned_countries["countries_en"].head(5)

/tmp/ipykernel_80869/4060907004.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned_countries["countries_en"] = df_cleaned_countries["countries_en"].str.split(",").str[0].str.strip()


1     France
3    Germany
5     Brazil
6    Germany
7     Brazil
Name: countries_en, dtype: object

In [82]:
# dropouts : removing rows with no countries information even after preprocessing

df_clean = df_cleaned_countries.dropna(subset=["countries"])

In [83]:
#dropout: removing rows with no product_name

df_clean = df_clean.dropna(subset = ["product_name"])

In [84]:
# checking final df shape & % of completeness

df_clean.shape

(2720373, 24)

In [123]:
# Calcul du % de NaN par colonne dans le data set "final"
pct_nan_clean = df_clean[columns_kept1].isna().mean() * 100

# Affichage avec arrondi à 2 décimales
print(pct_nan_clean.round(2))

code                           0.00
product_name                   1.04
pnns_groups_2                  0.00
brands                        34.88
quantity                      68.31
image_small_url               24.71
energy-kcal_100g               0.00
carbohydrates_100g             0.00
proteins_100g                  0.00
fat_100g                       0.00
first_packaging_code_geo      97.42
emb_codes_tags                94.42
emb_codes_tags                94.42
countries                      0.00
countries_en                   0.00
countries_tags                 0.00
origins                       96.09
origins_en                    96.09
origins_tags                  96.09
manufacturing_places          94.28
manufacturing_places_tags     94.28
first_packaging_code_geo      97.42
cities                       100.00
cities_tags                   97.22
purchase_places               94.57
stores                        86.08
dtype: float64


## (2)💈Preprocessing

### (2.1) 🔢 Numerical values: Scaling

In [ ]:
# numerical values already scaled (per 100g) - hence no outliers /  to be confirmed if required to do more

### (2.2) 🔡 Categorical values: Encoding

#### (2.2.1) 🥇🥈🥉 Ordinal Encoding

In [ ]:
# ordinal encoding to preserve "ranking" dimension (far => close ) consider 0 : ile de france, 1: france, 2: europe 3: monde
# TBD if required
# other option could be simply transforming data within a given radius (if longitude/latitude available) into 0, other with france as country into 1, european countries into 2, other countries into 3

from sklearn.preprocessing import OrdinalEncoder

encoder = OrdinalEncoder()

categories = ["0","1","2","3"]
df_encoded = df_clean.fit_transform()


# Define the ordinal mapping
distance_mapping = {'far': 1, 'medium': 2, 'close': 3}

# Apply the mapping to the column
data['distance_encoded'] = data['distance'].map(distance_mapping)

# Result
print(data)

#### (2.2.2) 🐱🐶🦊 OneHot Encoding

#### (2.2.3) 🐱 Manual Encoding

In [116]:
eu_countries = ["Austria",
"Belgium",
"Bulgaria",
"Croatia",
"Cyprus",
"Czech Republic",
"Denmark",
"Estonia",
"Finland",
"Germany",
"Greece",
"Hungary",
"Ireland",
"Italy",
"Latvia",
"Lithuania",
"Luxembourg",
"Malta",
"Netherlands",
"Poland",
"Portugal",
"Romania",
"Slovakia",
"Slovenia",
"Spain",
"Sweden"]

In [120]:
#later improve with country_final based on cleaning

df_clean["country_code"] = df_clean["countries_en"].apply(lambda x: 1 if x == "France" else 2 if x in eu_countries else 3)

df_clean.head(3)

,code,product_name,quantity,brands,origins,origins_tags,origins_en,manufacturing_places,manufacturing_places_tags,emb_codes_tags,first_packaging_code_geo,cities,cities_tags,purchase_places,stores,countries,countries_tags,countries_en,pnns_groups_2,image_small_url,energy-kcal_100g,fat_100g,carbohydrates_100g,proteins_100g,country_code
0,63,CIABATTA OLIVE,NaN,EDEKA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en:fr,en:france,France,unknown,https://images.openfoodfacts.org/images/produc...,332.0,25.0,3.0,23.0,1
1,1,Weihnachtsgelee (selbstgemacht von Jutta),280gr. 320 Kapseln,Bodylab,Spanien,en:spain,Spain,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DE,en:germany,Germany,unknown,https://images.openfoodfacts.org/images/produc...,374.0,2.0,7.0,79.0,2
2,2,Filets de poulet blanc x2,240-400 g,"SoLo, selbstgemacht 2 Liter","île d’Orléans,Québec,Canada","en:canada,en:quebec,fr:ile-d-orleans","Canada,Québec,fr:ile-d-orleans",Ancenis,ancenis,NaN,NaN,NaN,NaN,NaN,NaN,"Brasilien, Germany","en:brazil,en:germany",Brazil,unknown,https://images.openfoodfacts.org/images/produc...,141.0,2.7,0.9,30.0,3


In [119]:
df_clean["country_code"].unique()

array([1, 2, 3])

## (2.3) 🦾 Final Preprocessor

## (2.4)  Exporting & loading model

In [10]:
# extracting cleaned data set in CSV

df_clean.to_csv("open_food_df_clean.csv", sep="\t", index=False, encoding="utf-8")

NameError: name 'df_clean' is not defined

In [12]:
df_clean = pd.read_csv("open_food_df_clean.csv", sep="\t", encoding="utf-8",nrows = 500000)

/tmp/ipykernel_16497/409360579.py:1: DtypeWarning: Columns (10,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df_clean = pd.read_csv("open_food_df_clean.csv", sep="\t", encoding="utf-8",nrows = 500000)


In [13]:
df_clean.shape

(500000, 25)

## (3) 🤖 Modelling

In [16]:
# mini dataset download
df_model = pd.read_csv("open_food_df_clean.csv", on_bad_lines='skip' , sep="\t", nrows = 500000)

/tmp/ipykernel_26959/406424858.py:2: DtypeWarning: Columns (10,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df_model = pd.read_csv("open_food_df_clean.csv", on_bad_lines='skip' , sep="\t", nrows = 500000)


In [131]:
#df_model = df_clean
df_model["country_code"]

0         1
1         2
2         3
3         2
4         3
         ..
499995    3
499996    2
499997    3
499998    3
499999    3
Name: country_code, Length: 500000, dtype: int64

## (3.1) Level 1 MVP output

In [ ]:
# get recipe ingredients for the dataset

list_ingredients

# filter 1 : less kCal
# filter 2: lowest between (1: France, 2: UE, 3: World)

# par la suite peut être remplacé par Kmean puis DBScan etc..

['olive oil',
 'yellow onion',
 'celery rib',
 'carrot chopped',
 'garlic salt',
 'black pepper',
 'tomato paste',
 'dry red wine',
 'beef broth',
 'crushed tomatoes',
 'heavy cream',
 'fresh basil',
 'parmesan cheese']

In [ ]:
["energy-kcal_100g", "carbohydrates_100g", "proteins_100g", "fat_100g"]

1627695

In [ ]:
# Merci CLAUDE.AI

def find_best_match_for_ingredient(args):
    """
    Fonction pour traiter un seul ingrédient avec thefuzz
    """
    ingredient, df_model = args

    # Récupérer tous les noms de produits
    product_names = df_model['product_name'].tolist()

    # thefuzz.process.extractOne retourne (matched_string, score) - pas d'index
    match_result = process.extractOne(
        ingredient,
        product_names,
        scorer=fuzz.ratio
    )

    # Vérifier le score minimum
    if not match_result or match_result[1] < 80:
        return pd.DataFrame()  # Pas de match suffisant

    matched_name, score = match_result
    # Trouver l'index correspondant
    index = product_names.index(matched_name)

    # Récupérer la ligne correspondante
    matched_row = df_model.iloc[index].copy()

    # Créer le résultat pour cet ingrédient
    result_row = {
        'searched_ingredient': ingredient,
        'row_id': matched_row.name,  # Index original
        'product_name': matched_row['product_name'],
        'matched_product': matched_name,
        'match_score': score,
        'energy-kcal_100g': matched_row.get('energy-kcal_100g', 0),
        'carbohydrates_100g': matched_row.get('carbohydrates_100g', 0),
        'proteins_100g': matched_row.get('proteins_100g', 0),
        'fat_100g': matched_row.get('fat_100g', 0),
        "country_code": matched_row.get("country_code", 0)
    }

    return pd.DataFrame([result_row])

def optimized_ingredient_matching(list_ingredients, df_model, max_workers=None):
    """
    Version optimisée avec thefuzz et parallélisation
    """

    # Déterminer le nombre optimal de workers
    if max_workers is None:
        max_workers = min(32, len(list_ingredients), (os.cpu_count() or 1) * 2)

    print(f"🚀 Traitement de {len(list_ingredients)} ingrédients avec {max_workers} workers...")

    # Préparation des arguments pour la parallélisation
    args_list = [(ingredient, df_model) for ingredient in list_ingredients]

    all_results = []

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Soumettre toutes les tâches
        future_to_ingredient = {
            executor.submit(find_best_match_for_ingredient, args): args[0]
            for args in args_list
        }

        # Suivi de progression
        completed = 0
        total = len(future_to_ingredient)

        # Récupérer les résultats au fur et à mesure
        for future in as_completed(future_to_ingredient):
            completed += 1
            ingredient = future_to_ingredient[future]

            try:
                result_df = future.result()
                if not result_df.empty:
                    all_results.append(result_df)
                    print(f"✅ {completed:3d}/{total} - {ingredient} (score: {result_df.iloc[0]['match_score']:.0f})")
                else:
                    print(f"❌ {completed:3d}/{total} - {ingredient} (pas de match >= 80)")

            except Exception as exc:
                print(f"💥 {completed:3d}/{total} - Erreur pour {ingredient}: {exc}")

    # Concaténation et tri final
    if all_results:
        final_result = pd.concat(all_results, ignore_index=True)

        # Tri par score décroissant, puis par kcal croissant
        # (comme dans votre code original)
        final_result = final_result.sort_values(
            ['match_score', "country_code", 'energy-kcal_100g'],
            ascending=[False, True, True]
        )

        print(f"\n🎉 TERMINÉ! {len(final_result)} matches trouvés sur {len(list_ingredients)} ingrédients")
        print(f"Score moyen: {final_result['match_score'].mean():.1f}")

        return final_result
    else:
        print("\n😞 Aucun match trouvé avec score >= 80")
        return pd.DataFrame()

In [17]:
result_df = optimized_ingredient_matching(list_ingredients, df_model)
print(result_df)

🚀 Traitement de 13 ingrédients avec 13 workers...
✅   1/13 - tomato paste (score: 100)
✅   2/13 - heavy cream (score: 100)
✅   3/13 - olive oil (score: 100)
✅   4/13 - garlic salt (score: 100)
✅   5/13 - beef broth (score: 100)
✅   6/13 - crushed tomatoes (score: 100)
✅   7/13 - black pepper (score: 100)
✅   8/13 - parmesan cheese (score: 100)
✅   9/13 - yellow onion (score: 96)
❌  10/13 - celery rib (pas de match >= 80)
❌  11/13 - carrot chopped (pas de match >= 80)
✅  12/13 - fresh basil (score: 100)
✅  13/13 - dry red wine (score: 80)

🎉 TERMINÉ! 11 matches trouvés sur 13 ingrédients
Score moyen: 97.8
   searched_ingredient  row_id      product_name   matched_product  \
3          garlic salt   13315       Garlic salt       Garlic salt   
4           beef broth   14785        Beef broth        Beef broth   
6         black pepper  149981      Black pepper      Black pepper   
9          fresh basil  419766       Fresh Basil       Fresh Basil   
5     crushed tomatoes    7124  Crushe

In [ ]:
# origin of ingredients (France/ UE / World)
# kCal data

# df:
## list of ingredients
## columns : name, origin, kCal, distance category

### (3.1) 🐥 Logistic Regression & Classification Metrics

### (3.2) 🚀 Trying out all the models

### (3.3) 🧪 Fine-tuning a model